In [1]:
from UTILITY_quickstart import *
from scipy.optimize import minimize

#Start from golden lattice
tao = initializeTao(
    loadDefaultLatticeTF = False
)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Not using setLattice(). Golden lattice
WARNING! No beam file is specified!
Number of macro particles defined by input file


In [2]:
tao.ele_twiss("DTOTR")

{'mode_flip': False,
 'beta_a': 15.8379471946039,
 'alpha_a': -2.34734701102762,
 'gamma_a': 0.411040516184971,
 'phi_a': 77.0415206052783,
 'eta_a': 1.70374229029022e-05,
 'etap_a': 3.35821321449544e-06,
 'beta_b': 9.20171414252986,
 'alpha_b': 1.41094515314238,
 'gamma_b': 0.325022727162628,
 'phi_b': 62.4262150531777,
 'eta_b': 0.0559202303727423,
 'etap_b': 0.00600001800006481,
 'eta_x': 1.70374229029037e-05,
 'etap_x': 3.35821321449571e-06,
 'eta_y': 0.0559202303727423,
 'etap_y': 0.00600001800006481}

In [3]:
tao.ele_orbit("DTOTR")

{'x': -4.05050137830853e-17,
 'px': -3.15417949033432e-18,
 'y': 4.78224976711585e-16,
 'py': 5.13611857756645e-17,
 'z': -1.62920498062289e-15,
 'pz': 8.58306885885983e-15,
 'spin': array([0., 0., 0.]),
 'field': array([0., 0.]),
 'phase': array([0., 0.]),
 's': 1016.16282888229,
 't': 3.38957134967123e-06,
 'charge': 0.0,
 'dt_ref': 0.0,
 'p0c': 9999999986.94688,
 'beta': 0.9999999986944,
 'ix_ele': 1581,
 'state': 'Alive',
 'direction': 1,
 'species': 'Electron',
 'location': 'Downstream_End'}

In [4]:
getMatrix(tao, "MFFF", "DTOTR", print = True);

1.390362,-7.090993,0.000000,-0.000000,0.000000,0.000000
0.204478,-0.323622,0.000000,-0.000000,0.000000,-0.000000
-0.000000,-0.000000,0.705156,-1.681051,0.000000,0.055920
-0.000000,-0.000000,-0.203248,1.902658,0.000000,0.006000
0.000000,-0.000000,-0.015597,0.116483,1.000000,-0.000006
0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [5]:
tao.cmd(f"set ele XTCAVF VOLTAGE = 10e6")
tao.cmd(f"set ele XTCAVF PHI0 = {0 / 360.0}")
tao.ele_twiss("DTOTR")

{'mode_flip': False,
 'beta_a': 15.8379471663407,
 'alpha_a': -2.34734700688038,
 'gamma_a': 0.411040515689154,
 'phi_a': 77.0415206044066,
 'eta_a': -0.126558144736379,
 'etap_a': -0.0105760467869348,
 'beta_b': 9.20171414252992,
 'alpha_b': 1.41094515314238,
 'gamma_b': 0.325022727162625,
 'phi_b': 62.4262150531777,
 'eta_b': 0.0559202303727423,
 'etap_b': 0.00600001800006481,
 'eta_x': -0.126558144736379,
 'etap_x': -0.0105760467869348,
 'eta_y': 0.0559202303727423,
 'etap_y': 0.00600001800006481}

In [6]:
getMatrix(tao, "MFFF", "DTOTR", print = True);

1.390362,-7.090993,0.000000,-0.000000,-5.568421,-0.000000
0.204478,-0.323622,0.000000,-0.000000,-0.465420,-0.000000
0.027486,0.083781,0.705156,-1.681051,0.000547,0.055920
0.002949,0.008989,-0.203248,1.902658,0.000059,0.006000
-0.000003,-0.000009,-0.015597,0.116483,1.000000,-0.000006
0.491517,1.498225,-0.000000,0.000000,0.009775,1.000000


In [7]:
getMatrix(tao, "MFFF", "DTOTR")[0,4]

-5.56842063015725

## R15 maximization

Let's start by doing the simplest thing: maximizing the R15 term. This won't give us maximum effective resolution, just for the zero-emittance limit. Next step will be deconvolving out emittance contribution

In [8]:
#S20 configurator TCAV optics: https://docs.google.com/presentation/d/1pEDyfRYzVhJpeqzpDUfJOKxTxE6_achOUj4SHY-MoQQ/edit#slide=id.g2f39098462d_0_5

setQuadkG(tao, "Q5FF", -192)
setQuadkG(tao, "Q4FF", -162)
setQuadkG(tao, "Q3FF", 105)
setQuadkG(tao, "Q2FF", 120)
setQuadkG(tao, "Q1FF", -209)
setQuadkG(tao, "Q0FF", 120)
setQuadkG(tao, "Q0D", -139)
setQuadkG(tao, "Q1D", 225)
setQuadkG(tao, "Q2D", -139)

getMatrix(tao, "MFFF", "DTOTR", print = True);

-0.503226,-8.194331,0.000000,-0.000000,-11.561386,-0.000000
0.095960,-0.424607,0.000000,-0.000000,-0.869603,-0.000000
0.086511,0.123962,-0.057628,2.100488,0.000547,0.055920
0.009282,0.013301,-0.497087,0.765700,0.000059,0.006000
-0.000009,-0.000013,-0.027451,0.030215,1.000000,-0.000006
1.547036,2.216771,-0.000000,0.000000,0.009775,1.000000


In [9]:
def objective(params, tao):
    Q5FFkG, Q4FFkG, Q3FFkG, Q2FFkG, Q1FFkG, Q0FFkG, Q0DkG, Q1DkG, Q2DkG  = params
    
    try:
        #Prevent recalculation until changes are made
        tao.cmd("set global lattice_calc_on = F")
        
        setQuadkG(tao, "Q5FF", Q5FFkG)
        setQuadkG(tao, "Q4FF", Q4FFkG)
        setQuadkG(tao, "Q3FF", Q3FFkG)
        setQuadkG(tao, "Q2FF", Q2FFkG)
        setQuadkG(tao, "Q1FF", Q1FFkG)
        setQuadkG(tao, "Q0FF", Q0FFkG)
        setQuadkG(tao, "Q0D", Q0DkG)
        setQuadkG(tao, "Q1D", Q1DkG)
        setQuadkG(tao, "Q2D", Q2DkG)
        
        #Reenable lattice calculations
        tao.cmd("set global lattice_calc_on = T")
    
    except: #If Bmad doesn't like the proposed solution, don't crash, give a bad number
        return 1e20


    
    return -1 * abs ( getMatrix(tao, "MFFF", "DTOTR")[0,4] ) 


def solver(
    tao,
    verbose = False
):


    quadNameList = ["Q5FF", "Q4FF", "Q3FF", "Q2FF", "Q1FF", "Q0FF", "Q0D", "Q1D", "Q2D"] 
    initialGuess = [getQuadkG(tao, name) for name in quadNameList]

    #For now, just hardcoding bounds... could generalize if required
    #From "bounds.yml" as of 2025-01-10-11-11-35
    # Q5FFkGBounds: (-256, 0)  #BCON = -70
    # Q4FFkGBounds: (-446, 0)  #BCON = -71
    # Q3FFkGBounds: (0, 457)   #BCON = 106
    # Q2FFkGBounds: (0, 167)   #BCON = 112
    # Q1FFkGBounds: (-257, 0)  #BCON = -225
    # Q0FFkGBounds: (0, 167)   #BCON = 112
    # Q0DkGBounds : (-239, 0)  #BCON = -112
    # Q1DkGBounds : (0, 386)   #BCON = 177
    # Q2DkGBounds : (-223, 0)  #BCON = -112

    bounds = [(-256,0), (-446,0), (0,457), (0,167), (-257,0), (0,167), (-239,0), (0,386), (-223,0)]


    # Perform optimization using Nelder-Mead
    result = minimize(
        objective, 
        initialGuess, 
        method='Nelder-Mead',
        bounds = bounds,
        args = (tao)
    )


    # #Apply best result to the lattice
    # betaSetX, alphaSetX, betaSetY, alphaSetY = result.x
    
    # #Prevent recalculation until changes are made
    # tao.cmd("set global lattice_calc_on = F")
    
    # tao.cmd(f"set element beginning beta_a = {betaSetX}")
    # tao.cmd(f"set element beginning alpha_a = {alphaSetX}")
    # tao.cmd(f"set element beginning beta_b = {betaSetY}")
    # tao.cmd(f"set element beginning alpha_b = {alphaSetY}")
    
    # #Reenable lattice calculations
    # tao.cmd("set global lattice_calc_on = T")

    if verbose:
        print("Optimization Results:")
        print(f"Optimal Parameters: {result.x}")
        print(f"Objective Function Value at Optimal Parameters: {result.fun}")
        print(f"Number of Iterations: {result.nit}")
        print(f"Converged: {result.success}")

    quadVariableNameList = ["Q5FFkG", "Q4FFkG", "Q3FFkG", "Q2FFkG", "Q1FFkG", "Q0FFkG", "Q0DkG", "Q1DkG", "Q2DkG"] 

    
    
    return { quadVariableNameList[i] : result.x[i] for i in range(len(quadVariableNameList)) }

solver(tao, verbose = True)

Optimization Results:
Optimal Parameters: [-255.52877395    0.          189.78997461    0.         -257.
    0.         -239.          386.         -223.        ]
Objective Function Value at Optimal Parameters: -356.746011439694
Number of Iterations: 117
Converged: True


{'Q5FFkG': -255.52877394716324,
 'Q4FFkG': 0.0,
 'Q3FFkG': 189.78997460985195,
 'Q2FFkG': 0.0,
 'Q1FFkG': -257.0,
 'Q0FFkG': 0.0,
 'Q0DkG': -239.0,
 'Q1DkG': 386.0,
 'Q2DkG': -223.0}

## Beta function ratio maximization

In [31]:
#Start from golden lattice
tao = initializeTao(
    loadDefaultLatticeTF = False
)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Not using setLattice(). Golden lattice
WARNING! No beam file is specified!
Number of macro particles defined by input file


In [32]:
betaX = tao.ele_twiss("DTOTR")["beta_a"] 
R15  = getMatrix(tao, "MFFF", "DTOTR")[0,4]

print(f"betaX = {betaX} m. R15 = {R15}")

assumeEmittance = 20e-6
assumeBunchLength = 20e-6

emittanceContrib = np.sqrt( betaX * (assumeEmittance / ( 10e6 / 0.511) ) )
zContrib = R15 * assumeBunchLength
estimatedSpot = np.sqrt( emittanceContrib ** 2 + zContrib ** 2)

print(emittanceContrib)
print(zContrib)
print(estimatedSpot)


betaX = 15.8379471946039 m. R15 = 0.0
4.023230298265958e-06
0.0
4.023230298265958e-06


In [33]:
tao.cmd(f"set ele XTCAVF VOLTAGE = 10e6")
tao.cmd(f"set ele XTCAVF PHI0 = {0 / 360.0}")




betaX = tao.ele_twiss("DTOTR")["beta_a"] 
R15  = getMatrix(tao, "MFFF", "DTOTR")[0,4]

print(f"betaX = {betaX} m. R15 = {R15}")

assumeEmittance = 20e-6
assumeBunchLength = 20e-6

emittanceContrib = np.sqrt( betaX * (assumeEmittance / ( 10e6 / 0.511) ) )
zContrib = R15 * assumeBunchLength
estimatedSpot = np.sqrt( emittanceContrib ** 2 + zContrib ** 2)

print(emittanceContrib)
print(zContrib)
print(estimatedSpot)

betaX = 15.8379471663407 m. R15 = -5.56842063015725
4.023230294676182e-06
-0.000111368412603145
0.0001114410593441589


In [34]:
def getEstimatedSpotRatio():      
        assumeEmittance = 20e-6
        assumeBunchLength = 20e-6


        tao.cmd(f"set ele XTCAVF VOLTAGE = 0")

        betaX = tao.ele_twiss("DTOTR")["beta_a"] 
        R15  = getMatrix(tao, "MFFF", "DTOTR")[0,4]
        
        emittanceContrib = np.sqrt( betaX * (assumeEmittance / ( 10e6 / 0.511) ) )
        zContrib = R15 * assumeBunchLength
        estimatedSpotOFF = np.sqrt( emittanceContrib ** 2 + zContrib ** 2)

       
    



        tao.cmd(f"set ele XTCAVF VOLTAGE = 10e6")

        betaX = tao.ele_twiss("DTOTR")["beta_a"] 
        R15  = getMatrix(tao, "MFFF", "DTOTR")[0,4]
        
        emittanceContrib = np.sqrt( betaX * (assumeEmittance / ( 10e6 / 0.511) ) )
        zContrib = R15 * assumeBunchLength
        estimatedSpotON = np.sqrt( emittanceContrib ** 2 + zContrib ** 2)

        return estimatedSpotON / estimatedSpotOFF

In [36]:
getEstimatedSpotRatio()

27.699398513724365

In [37]:
#S20 configurator TCAV optics: https://docs.google.com/presentation/d/1pEDyfRYzVhJpeqzpDUfJOKxTxE6_achOUj4SHY-MoQQ/edit#slide=id.g2f39098462d_0_5

setQuadkG(tao, "Q5FF", -192)
setQuadkG(tao, "Q4FF", -162)
setQuadkG(tao, "Q3FF", 105)
setQuadkG(tao, "Q2FF", 120)
setQuadkG(tao, "Q1FF", -209)
setQuadkG(tao, "Q0FF", 120)
setQuadkG(tao, "Q0D", -139)
setQuadkG(tao, "Q1D", 225)
setQuadkG(tao, "Q2D", -139)

getEstimatedSpotRatio()

56.4734678003475

In [38]:
def objective(params, tao):
    Q5FFkG, Q4FFkG, Q3FFkG, Q2FFkG, Q1FFkG, Q0FFkG, Q0DkG, Q1DkG, Q2DkG  = params
    
    try:
        #Prevent recalculation until changes are made
        tao.cmd("set global lattice_calc_on = F")
        
        setQuadkG(tao, "Q5FF", Q5FFkG)
        setQuadkG(tao, "Q4FF", Q4FFkG)
        setQuadkG(tao, "Q3FF", Q3FFkG)
        setQuadkG(tao, "Q2FF", Q2FFkG)
        setQuadkG(tao, "Q1FF", Q1FFkG)
        setQuadkG(tao, "Q0FF", Q0FFkG)
        setQuadkG(tao, "Q0D", Q0DkG)
        setQuadkG(tao, "Q1D", Q1DkG)
        setQuadkG(tao, "Q2D", Q2DkG)

        
        #Reenable lattice calculations
        tao.cmd("set global lattice_calc_on = T")


    
    except: #If Bmad doesn't like the proposed solution, don't crash, give a bad number
        return 1e20


    
    return -1 * ( getEstimatedSpotRatio() ) 


def solver(
    tao,
    verbose = False
):


    quadNameList = ["Q5FF", "Q4FF", "Q3FF", "Q2FF", "Q1FF", "Q0FF", "Q0D", "Q1D", "Q2D"] 
    initialGuess = [getQuadkG(tao, name) for name in quadNameList]

    #For now, just hardcoding bounds... could generalize if required
    #From "bounds.yml" as of 2025-01-10-11-11-35
    # Q5FFkGBounds: (-256, 0)  #BCON = -70
    # Q4FFkGBounds: (-446, 0)  #BCON = -71
    # Q3FFkGBounds: (0, 457)   #BCON = 106
    # Q2FFkGBounds: (0, 167)   #BCON = 112
    # Q1FFkGBounds: (-257, 0)  #BCON = -225
    # Q0FFkGBounds: (0, 167)   #BCON = 112
    # Q0DkGBounds : (-239, 0)  #BCON = -112
    # Q1DkGBounds : (0, 386)   #BCON = 177
    # Q2DkGBounds : (-223, 0)  #BCON = -112

    bounds = [(-256,0), (-446,0), (0,457), (0,167), (-257,0), (0,167), (-239,0), (0,386), (-223,0)]


    # Perform optimization using Nelder-Mead
    result = minimize(
        objective, 
        initialGuess, 
        method='Nelder-Mead',
        bounds = bounds,
        args = (tao)
    )


    # #Apply best result to the lattice
    # betaSetX, alphaSetX, betaSetY, alphaSetY = result.x
    
    # #Prevent recalculation until changes are made
    # tao.cmd("set global lattice_calc_on = F")
    
    # tao.cmd(f"set element beginning beta_a = {betaSetX}")
    # tao.cmd(f"set element beginning alpha_a = {alphaSetX}")
    # tao.cmd(f"set element beginning beta_b = {betaSetY}")
    # tao.cmd(f"set element beginning alpha_b = {alphaSetY}")
    
    # #Reenable lattice calculations
    # tao.cmd("set global lattice_calc_on = T")

    if verbose:
        print("Optimization Results:")
        print(f"Optimal Parameters: {result.x}")
        print(f"Objective Function Value at Optimal Parameters: {result.fun}")
        print(f"Number of Iterations: {result.nit}")
        print(f"Converged: {result.success}")

    quadVariableNameList = ["Q5FFkG", "Q4FFkG", "Q3FFkG", "Q2FFkG", "Q1FFkG", "Q0FFkG", "Q0DkG", "Q1DkG", "Q2DkG"] 

    
    
    return { quadVariableNameList[i] : result.x[i] for i in range(len(quadVariableNameList)) }

solver(tao, verbose = True)

Optimization Results:
Optimal Parameters: [-256.         -172.25416034    2.74860979  126.51008991 -201.74835159
  127.12021983 -150.17414119  234.72993228 -161.48889705]
Objective Function Value at Optimal Parameters: -251.29405428803557
Number of Iterations: 1194
Converged: False


{'Q5FFkG': -256.0,
 'Q4FFkG': -172.2541603444513,
 'Q3FFkG': 2.748609788867163,
 'Q2FFkG': 126.51008991158068,
 'Q1FFkG': -201.7483515852437,
 'Q0FFkG': 127.12021983329481,
 'Q0DkG': -150.1741411865047,
 'Q1DkG': 234.7299322790954,
 'Q2DkG': -161.4888970463495}